# 04 — Regression (Supervised) for PM2.5 (Tabular Time-Lag Features)
Mục tiêu:
- Biến bài toán chuỗi thời gian thành bài toán **hồi quy có giám sát**: dự đoán PM2.5(t+h) từ đặc trưng tại thời điểm t.
- Thấy rõ: **leakage** và vì sao phải split theo thời gian.
- So sánh tư duy hồi quy (feature-based) vs ARIMA (time-series-based).


In [ ]:
import os
os.makedirs('images', exist_ok=True)
from pathlib import Path

# ===== PARAMETERS =====
USE_UCIMLREPO = False

# Path to the raw ZIP (relative to project root)
RAW_ZIP_PATH = 'data/raw/PRSA2017_Data_20130301-20170228.zip'

def resolve_project_root(raw_zip_rel: str) -> Path:
    """Resolve project root robustly for both Jupyter and Papermill runs."""
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent]
    root = cwd
    for _ in range(3):
        candidates.append(root)
        root = root.parent
    for r in candidates:
        if (r / raw_zip_rel).exists():
            return r
    return cwd

PROJECT_ROOT = resolve_project_root(RAW_ZIP_PATH)
RAW_ZIP_ABS = str((PROJECT_ROOT / RAW_ZIP_PATH).resolve())

LAG_HOURS = [1, 3, 24]
HORIZON = 1
TARGET_COL = 'PM2.5'
OUTPUT_REG_DATASET_PATH = 'data/processed/dataset_for_regression.parquet'
CUTOFF = '2017-01-01'
MODEL_OUT = 'regressor.joblib'
METRICS_OUT = 'regression_metrics.json'
PRED_SAMPLE_OUT = 'regression_predictions_sample.csv'


In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.classification_library import Paths
from src.regression_library import run_prepare_regression_dataset, run_train_regression
paths = Paths(project_root=PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)
print('RAW_ZIP_ABS =', RAW_ZIP_ABS)


## 1) Tạo dataset hồi quy (lag features + time features + y = PM2.5(t+h))


In [ ]:
out_path = run_prepare_regression_dataset(
    paths=paths,
    use_ucimlrepo=USE_UCIMLREPO,
    raw_zip_path=RAW_ZIP_ABS,
    lag_hours=LAG_HOURS,
    horizon=HORIZON,
    target_col=TARGET_COL,
)
print('Saved:', out_path)


## 2) Quick EDA cho dataset hồi quy


In [ ]:
ds_path = (PROJECT_ROOT / OUTPUT_REG_DATASET_PATH).resolve()
df = pd.read_parquet(ds_path)
print(df.shape)
display(df.head())
missing = df.isna().mean().sort_values(ascending=False).head(15)
display(missing)
plt.figure()
pd.Series(df[TARGET_COL]).dropna().plot(kind='hist', bins=60)
plt.title(f'Distribution of {TARGET_COL} (raw)')
plt.xlabel('PM2.5')
plt.ylabel('Frequency')
plt.savefig('images/regression_target_dist.png')
plt.show()


## 3) Train/Test theo thời gian + train regressor


In [ ]:
out = run_train_regression(
    paths=paths,
    cutoff=CUTOFF,
    model_out=MODEL_OUT,
    metrics_out=METRICS_OUT,
    preds_out=PRED_SAMPLE_OUT,
)
print('Metrics:')
print(json.dumps(out['metrics'], ensure_ascii=False, indent=2))
pred_df = out['pred_df']
display(pred_df.head())
# Plot a small window for storytelling
sample = pred_df.dropna().iloc[:500].copy()
plt.figure(figsize=(10,4))
plt.plot(sample['datetime'], sample['y_true'], label='Actual')
plt.plot(sample['datetime'], sample['y_pred'], label='Predicted')
plt.title('Regression: Actual vs Predicted (sample)')
plt.xlabel('Date')
plt.ylabel('PM2.5 Concentration (ug/m3)')
plt.legend()
plt.tight_layout()
plt.savefig('images/regression_pred_sample.png')
plt.show()


# Q2: Regression Baseline Analysis
**Thực hành Q2:**
1. **Ý nghĩa Lag 24h**: Do đặc tính sinh hoạt hàng ngày (giờ cao điểm, thấp điểm lặp lại) và chu kỳ thời tiết (ngày/đêm) nên PM2.5 thường có tương quan mạnh với chính nó cách đây 24h.
2. **Cutoff vs Shuffle**: Dữ liệu chuỗi thời gian có tính thứ tự. Shuffle sẽ gây **leakage** (mô hình học từ tương lai để dự đoán quá khứ gần). Cutoff đảm bảo mô hình chỉ biết dữ liệu lịch sử.
3. **RMSE vs MAE**: RMSE phạt nặng các sai số lớn (outliers/spikes), MAE đánh giá trung bình sai số tuyệt đối. Khi RMSE >> MAE, chứng tỏ mô hình dự báo sai lệch nhiều ở các điểm đột biến (spike).

In [ ]:
# Plotting errors to enable visual analysis
if 'pred_df' in locals():
    pred_df['error'] = pred_df['y_true'] - pred_df['y_pred']
    plt.figure(figsize=(15, 6))
    plt.plot(pred_df['datetime'], pred_df['error'], label='Residual (Error)', alpha=0.7)
    plt.axhline(0, color='r', linestyle='--')
    plt.title('Residuals over Time')
    plt.xlabel('Date')
    plt.ylabel('Residual Error (Actual - Pred)')
    plt.legend()
    plt.savefig('images/regression_residuals.png')
    plt.show()
    
    # Check specific large error points
    worst_errors = pred_df.iloc[pred_df['error'].abs().argsort()[::-1]].head(5)
    print("Top 5 worst predictions:")
    display(worst_errors)